In [1]:
using Random, Gnuplot, Statistics

┌ Info:   Gnuplot version: 5.2.0
└ @ Gnuplot C:\Users\User\.julia\packages\Gnuplot\GkEY3\src\Gnuplot.jl:111


In [2]:
X = randn(1000) * 10e-6
@gp "set multiplot layout 1,2"
@gp :- 1 X
h = hist(X, nbins=10)
@gp :- 2 h 

┌ Info: Creating session default...
└ @ Gnuplot C:\Users\User\.julia\packages\Gnuplot\GkEY3\src\Gnuplot.jl:173
┌ Info:   Gnuplot version: 5.2.0
└ @ Gnuplot C:\Users\User\.julia\packages\Gnuplot\GkEY3\src\Gnuplot.jl:111


GNUPLOT (default) -> wxt
GNUPLOT (default) -> 0 enhanced
GNUPLOT (default) -> multiplot> plot  
GNUPLOT (default) ->                  ^
GNUPLOT (default) ->            line 0: function to plot expected


In [3]:
mean(X), std(X)

(-1.0970758440418569e-7, 9.644229019973018e-6)

In [3]:
function trapez(f, a, b, n)
    h = (b - a)/n
    result = 0.5*(f(a) + f(b))
    for i in 1:n-1
        result += f(a + i*h)
    end
    result * h
end

function rk4(f, x, y, h)
    k1 = h * f(x       , y        )
    k2 = h * f(x + 0.5h, y + 0.5k1)
    k3 = h * f(x + 0.5h, y + 0.5k2)
    k4 = h * f(x +    h, y +    k3)
    
    return y + (k1 + 2*(k2 + k3) + k4)/6.0
end

const ħ = 1.055e-34 # J*s 
const kB = 1.381e-23 # J*K-1
const g = 9.8 # m/s^2

const l1 = 76e-3 # m
const l2 = 113e-3 # m
const yh = 2.8e-3 # m
const d = 6e-6 # m
const b = 2e-6 # m
const a1 = 0.5(- d - b)
const a2 = 0.5(- d + b)
const b1 = 0.5(  d - b)
const b2 = 0.5(  d + b)

const m = 3.349e-26 # kg
const T = 2.5e-3 # K
const σv = sqrt(kB*T/m) # разброс скорости
const σ₀ = 10e-6 # 10 mkm # разброс по х и по у
const σz = 3e-4 # 0.3 mm # по z
const σk = m*σv / (ħ*√3) # 2e8 m/s # отклонение для волнового вектора

1.8608498440793237e8

In [4]:
v₀ = zeros(3)
k₀ = zeros(3);

In [194]:
v₀ = randn(3) * σv
k₀ = randn(3) * σk # m*v₀/ħ

3-element Array{Float64,1}:
 -3.649135770763831e8 
  1.7069564092337338e6
  2.70439434749401e8  

## Before slits

$$
\begin{aligned}
\psi_{x}\left(x, t ; k_{0 x}\right) &=\left(2 \pi s_{0}^{2}(t)\right)^{-1 / 4} \exp \left[-\frac{\left(x-v_{0 x} t\right)^{2}}{4 \sigma_{0} s_{0}(t)}+i k_{0 x}\left(x-v_{0 x} t\right)\right] \\
\rho_{x}(x, t) &=\left(2 \pi \varepsilon_{0}^{2}(t)\right)^{-1 / 2} \exp \left[-\frac{x^{2}}{2 \varepsilon_{0}^{2}(t)}\right]
\end{aligned}
$$

$s_{0}(t)=\sigma_{0} + \frac{i \hbar t}{2 m \sigma_{0}}$

 $\varepsilon_{0}^{2}(t)=s_{0}s_{0}^* + \left(\frac{\hbar t \sigma_{v}}{m}\right)^{2}$ 

In [5]:
ε₀(t) = σ₀^2 + ( ħ/(2m*σ₀) * t )^2 + (ħ*t*σv/m)^2
s₀(t) = σ₀ + im*ħ/(2m*σ₀) * t

ρₓ(x, t) = exp( -x^2 / (2ε₀(t)) ) / sqrt( 2π*ε₀(t) )

t₁(v, z) = sqrt( 2*(l1-z)/g + (v/g)^2 ) - v/g
t₁(0, 0) # 124 ms

0.1245399698154478

In [6]:
tf1 = t₁(v₀[3], 0) # s # время достижения первого экрана
#v₀[3]*tf1 + 0.5g*tf1^2 # m

0.1245399698154478

In [171]:
X1 = range(-100, stop = 100, length = 100)*1e-6 
Z1 = range(0, stop = l1, length = 100)
Cron1 = range(0, stop = tf1, length = 100)

P1 = [ ρₓ(x, t) for t in Cron1, x in X1 ]
P1 /= maximum(P1); # нормируем на единицу

In [94]:
@gp 1000000X1 P1[1,:] "with lines notit lw 2 lc rgb 'black'"

GNUPLOT (default) -> gnuplot> plot    $data0 with lines notit lw 2 lc rgb 'black'
GNUPLOT (default) ->                                                             ^
GNUPLOT (default) ->          line 318592: all points y value undefined!


In [17]:
@gsp 1000Z1 1000000X1 P1 "w pm3d notit"

In [172]:
@gp "set title 'Wavefun before the slits'" xlab="Z, mkm" ylab="X, mkm"
@gp :- 1000Z1 1000000X1 P1 "w image notit"

GNUPLOT (default) -> gnuplot> plot  
GNUPLOT (default) ->                ^
GNUPLOT (default) ->          line 680746: function to plot expected


## After slits

$$
K_{x}\left(x_{\beta}, t_{\beta} ; x_{\alpha}, t_{\alpha}\right)=\left(\frac{m}{2 i \pi \hbar\left(t_{\beta}-t_{\alpha}\right)}\right)^{1 / 2} \exp \frac{i m}{\hbar}\left(\frac{\left(x_{\beta}-x_{\alpha}\right)^{2}}{2\left(t_{\beta}-t_{\alpha}\right)}\right)
$$

$$
\psi_{x}\left(x, t ; k_{0 x}, k_{0 z}, z_{0}\right)=\psi_{A}+\psi_{B}
$$
$$
\begin{aligned}
\psi_{A} &=\int_{A} K_{x}\left(x, t ; x_{a}, t_{1}\left(v_{0 z}, z_{0}\right)\right) \psi_{x}\left(x_{a}, t_{1}\left(v_{0 z}, z_{0}\right) ; k_{0 x}\right) d x_{a} \\
\psi_{B} &=\int_{B} K_{x}\left(x, t ; x_{b}, t_{1}\left(v_{0 z}, z_{0}\right)\right) \psi_{x}\left(x_{b}, t_{1}\left(v_{0 z}, z_{0}\right) ; k_{0 x}\right) d x_{b}
\end{aligned}
$$

где $\psi_{x}\left(x_{a}, t_{1}\left(v_{0 z}, z_{0}\right) ; k_{0 x}\right)$ и $\psi_{x}\left(x_{b}, t_{1}\left(v_{0 z}, z_{0}\right) ; k_{0 x}\right)$ плотность вероятности равна
$$
\rho_{x}\left(x, t ; k_{0 z}, z_{0}\right)=\int_{-\infty}^{+\infty}\left(2 \pi \tau^{2}\right)^{-1 / 2} \exp \left(-\frac{k_{0 x}^{2}}{2 \tau^{2}}\right)\left|\psi_{x}\left(x, t ; k_{0 x}, k_{0 z}, z_{0}\right)\right|^{2} d k_{0 x}
$$
Время прихода $t_{2}$ центра волнового пакета на детектирующую пластину зависит от $z_{0}$ и $v_{0 z} .$ У нас есть $t_{2}=\sqrt {\frac{2\left(l_{1}+l_{2} - z_{0}\right)} {g}+\left(\frac{v_{0} z}{g}\right)^{2}}-\frac{v_{0 z}}{g} .$ Для $z_{0}=0$ и $v_{0 z}=0, t_{2}=\sqrt {\frac{2\left(l_{1}+l_{2}\right)}{g}}=196 \mathrm{ms}$
и атомы ускоряются, до $v_{z 2}=g t_{2}=1.93 \mathrm{m} / \mathrm{s}$

In [36]:
function ρₓ(xᵢ, tᵢ) 
    
    Kₓ(xb, tb, xa, ta) = sqrt( m/(2im*π*ħ*(tb-ta)) ) * exp( im*m*(xb-xa)^2 / (2ħ*(tb-ta)) )
    
    function subintrho(k) 

        ψₓ(x, t) = ( 2π*s₀(t) )^-0.25 * exp( -(x-ħ*k*t/m)^2 / (4σ₀*s₀(t)) + im*k*(x-ħ*k*t/m) )
        subintpsi(x) = Kₓ(xᵢ, tᵢ, x, tf1) * ψₓ(x, tf1)

        ψa = trapez(subintpsi, a1, a2, 200)
        ψb = trapez(subintpsi, b1, b2, 200)

        exp( -k^2 / (2σv^2) ) * abs2(ψa + ψb)
    end
    
    trapez(subintrho, -10σk, 10σk, 20) / sqrt( 2π*σv^2 )
end

ρₓ (generic function with 1 method)

In [7]:
t₂(v, z) = sqrt( 2*(l1+l2-z)/g + (v/g)^2 ) - v/g
t₂(0, 0) # 196 ms

0.19639610121239315

In [8]:
tf2 = t₂(v₀[3], 0) # s # время достижения второго экрана

0.19639610121239315

In [11]:
#X2 = range(-10, stop = 10, length = 200)*1e-6
X2 = range(-800, stop = 800, length = 200)*1e-6
Z2 = range(l1, stop = l2, length = 100)
Cron2 = range(tf1, stop = tf2, length = 100) ## +0.001 

0.1245399698154478:0.0007258195090600539:0.19639610121239315

In [40]:
@time P2 = [ ρₓ(x, t) for t in Cron2, x in X2 ];

103.760820 seconds (1.52 G allocations: 32.844 GiB, 4.49% gc time)


In [42]:
@gp 1000000X2 P2[1,:] "with lines notit lw 2 lc rgb 'black'"

In [41]:
@gsp 1000Z2 1000000X2 P2 "w pm3d notit"

# + speed, - readability

In [210]:
t2mm = sqrt( 2*(l1+2e-3)/g + (v₀[3]/g)^2 ) - v₀[3]/g

0.1261680123761121

In [56]:
Cron2mm = range(tf1, stop = t2mm, length = 100)

0.1245399698154478:1.6444874350144472e-5:0.1261680123761121

In [72]:
function fastamazafaka()
    
    A = ( 2π*s₀(tf1) )^-0.25
    
    function ρₓ(xᵢ, tᵢ) 
        
        B = sqrt( m/(2im*π*ħ*(tᵢ-tf1)) )
    
        Kₓ(xb, tb, xa, ta) = exp( im*m*(xb-xa)^2 / (2ħ*(tb-ta)) )

        function subintrho(k) 

            ψₓ(x, t) = exp( -(x-ħ*k*t/m)^2 / (4σ₀*s₀(t)) + im*k*(x-ħ*k*t/m) )
            subintpsi(x) = Kₓ(xᵢ, tᵢ, x, tf1) * ψₓ(x, tf1)

            ψa = trapez(subintpsi, a1, a2, 200)
            ψb = trapez(subintpsi, b1, b2, 200)

            exp( -k^2 / (2σv^2) ) * abs2(ψa + ψb)
        end

        abs2(A * B) * trapez(subintrho, -σk, σk, 20) / sqrt( 2π*σv^2 )
    end

    P3 = [ ρₓ(x, t) for t in Cron2, x in X2 ];
end

fastamazafaka (generic function with 1 method)

In [73]:
@time P3 = fastamazafaka();

 49.394490 seconds (1.52 G allocations: 32.801 GiB, 5.72% gc time)


In [74]:
P3 /= maximum(P3[4:end,:]);

In [185]:
@gp "set title 'Wavefun after the slits'" xlab="t, s" ylab="X, mkm"
@gp :- Cron2[4:end] 1000000X2 P3[4:end,:] "w image notit"

GNUPLOT (default) -> gnuplot> plot  
GNUPLOT (default) ->                ^
GNUPLOT (default) ->          line 761825: function to plot expected


In [50]:
@gsp P3[2:end,:] "w pm3d notit"

# Trajectories
## Before slits

$$
z(t)=v_{0 z} t+\frac{1}{2} g t^{2}+z_{0} \frac{\sigma_{z}(t)}{\sigma_{z}}
$$

$$
\begin{aligned}
x(t) &=v_{0 x} t+\sqrt{x_{0}^{2}+y_{0}^{2}} \frac{\sigma_{0}(t)}{\sigma_{0}} \cos \varphi(t) \\
y(t) &=v_{0 y} t+\sqrt{x_{0}^{2}+y_{0}^{2}} \frac{\sigma_{0}(t)}{\sigma_{0}} \sin \varphi(t)
\end{aligned}
$$

$\sigma_{0}^{2}(t)=\sigma_{0}^{2}+\left(\frac{\hbar t}{2 m \sigma_{0}}\right)^{2} .$

$\varphi(t)=\varphi_{0}+\arctan \left(-\frac{\hbar t}{2 m \sigma_{0}^{2}}\right)$

$ \cos \left(\varphi_{0}\right)=\frac{x_{0}}{\sqrt{x_{0}^{2}+y_{0}^{2}}}, $ 

$\sin \left(\varphi_{0}\right)=\frac{y_{0}}{\sqrt{x_{0}^{2}+y_{0}^{2}} } .$ 

$\sin(\arctan x) = \frac{x}{\sqrt{1+x^2}}$

$\cos(\arctan x) = \frac{1}{\sqrt{1+x^2}}$

$\cos(a+b) = \cos a\cos b - \sin a\sin b$

$\sin(a+b) = \sin a\cos b + \cos a\sin b$

In [117]:
function xbs(t, xo, yo, vo)
    
    cosϕ = xo / sqrt( xo^2 + yo^2 )
    sinϕ = yo / sqrt( xo^2 + yo^2 )
    atnx = -ħ*t / (2m*σ₀^2)
    sinatnx = atnx / sqrt( atnx^2 + 1)
    cosatnx =    1 / sqrt( atnx^2 + 1)
    cosϕt = cosϕ * cosatnx - sinϕ * sinatnx
    #sinϕt = sinϕ*cosatnx + cosϕ*sinatnx
    
    σ₀t = sqrt( σ₀^2 + ( ħ*t / (2*m*σ₀) )^2 )
    
    vo*t + sqrt(xo^2 + yo^2) * σ₀t/σ₀ * cosϕt
end

xbs (generic function with 1 method)

In [77]:
@gp "set title 'Trajectories before slits'" xlab="t, s" ylab="X, mkm" # "set yrange [-10:10]"

@time for i in 1:80
    
    x0 = randn()*σ₀
    y0 = randn()*σ₀
    v0 = randn()*σv*1e-4
    
    prtclᵢ = [ xbs(t,x0,y0,v0) for t in Cron1 ]
    
    @gp :- Cron1 1000000prtclᵢ "with lines notit lw 2 lc rgb 'black'"
    # slits -4-> -2, 2->4 mkm
end

GNUPLOT (default) -> gnuplot> plot  
GNUPLOT (default) ->                ^
GNUPLOT (default) ->          line 298308: function to plot expected
 18.081040 seconds (391.92 k allocations: 20.324 MiB, 0.08% gc time)


## After slits

the velocity $v_{x}(x, t)=\frac{\hbar}{m} \frac{I m\left(\frac{\partial \psi}{\partial x} \psi^{*}\right)}{\psi \psi^{*}}$ . We obtain:

$$
v_{x}(x, t)=\frac{1}{t-t_{1}}\left[x+\frac{-1}{2\left(\alpha^{2}+\beta_{t}^{2}\right)}\left(\beta_{t} I m\left(\frac{C(x, t)}{H(x, t)}\right)+\alpha \operatorname{Re}\left(\frac{C(x, t)}{H(x, t)}\right)-\beta_{t} \gamma_{x, t}\right)\right]
$$
with
$$
\begin{array}{c}
H(x, t)=\int_{X_{A}-b}^{X_{A}+b} f(x, u, t) d u+\int_{X_{B}-b}^{X_{B}+b} f(x, u, t) d u \\
C(x, t)=[f(x, u, t)]_{u=X_{A}-b}^{u=X_{A}+b}+[f(x, u, t)]_{u=X_{B}-b}^{u=X_{B}+b}
\end{array}
$$
where $X_{A}$ and $X_{B}$ are the centers of the two slits, and where
$$
f(x, u, t)=\exp \left[\left(\alpha+i \beta_{t}\right) u^{2}+i \gamma_{x, t} u\right]
$$
$$
\begin{array}{l}
\alpha=-\frac{1}{4 \sigma_{0}^{2}\left(1+\left(\frac{\hbar t_{1}}{2 m \sigma_{0}^{2}}\right)^{2}\right)} \\
\beta_{t}=\frac{m}{2 \hbar}\left(\frac{1}{t-t_{1}}+\frac{1}{t_{1}\left(1+\left(\frac{2 m \sigma_{0}^{2}}{\hbar t_{1}}\right)^{2}\right)}\right) \\
\gamma_{x, t}=-\frac{m x}{\hbar\left(t-t_{1}\right)}
\end{array}
$$


In [25]:
function Uₓ(t, x)
    
    γ = -m*x / ( ħ*(t-tf1) )
    β = m/(2ħ) * ( 1/(t-tf1) + 1 / ( tf1*( 1 + (2m*σ₀^2 / (ħ*tf1) )^2 ) ) )
    α = -( 4σ₀^2 * ( 1 + ( ħ*tf1 / (2m*σ₀^2) )^2 ) )^-1
    
    f(x, u, t) = exp( ( α + im*β )*u^2 + im*γ*u )
    
    C = f(x, a2, t) - f(x, a1, t) + f(x, b2, t) - f(x, b1, t)
    H = trapez( u-> f(x, u, t) , a1, a2, 400) + trapez( u-> f(x, u, t) , b1, b2, 400)
    
    return 1/(t-tf1) * ( x - 0.5/(α^2 + β^2) * ( β*imag(C/H) + α*real(C/H) - β*γ ) )
end

init() = bitrand()[1] ? rand()*(b2 - b1) + b1 : rand()*(a2 - a1) + a1
myrng(a, b, N) = collect( range(a, stop = b, length = N ÷ 2) )

myrng (generic function with 1 method)

In [212]:
n1 = 1
xx = 0
ξ = 0.00001
while xx < Cron2mm[end]-Cron2mm[1]
    n1+=1
    xx += (ξ*n1)^2 
end
n1

366

In [213]:
steps1 = [ (ξ*i)^2 for i in 1:n1 ]
steps1[1], steps1[end]

(1.0000000000000002e-10, 1.3395600000000004e-5)

In [214]:
Cronadapt2mm = accumulate(+, steps1, init = Cron2mm[1] )
Cronadapt2mm[1], Cronadapt2mm[end]

(0.1245399699154478, 0.12618093691544777)

In [26]:
function modelsolver(Np = 10, Nt = 100)
    
    #xo = [ init() for i = 1:Np ] # случайные
    xo = vcat( myrng(a2, a1, Np), myrng(b1, b2, Np) ) # равномерно
    
    xpath = zeros(Nt, Np)
    xpath[1,:] = xo
    
    for i in 2:Nt, j in 1:Np
        xpath[i,j] = rk4(Uₓ, Cronadapt[i], xpath[i-1,j], steps[i] )
    end
    
    xpath
end

modelsolver (generic function with 3 methods)

In [27]:
@time paths = modelsolver(20, n);

  3.834920 seconds (8.15 M allocations: 163.666 MiB, 0.59% gc time)


In [28]:
@gp "set title 'Trajectories after the slits'" xlab="t, s" ylab="X, mkm" "set yrange [-800:800]"
for i in 1:size(paths, 2)
    @gp :- Cronadapt 1e6paths[:,i] "with lines notit lw 1 lc rgb 'black'"
end

GNUPLOT (default) -> gnuplot> plot  
GNUPLOT (default) ->                ^
GNUPLOT (default) ->          line 4114: function to plot expected


# 3D

In [12]:
n = 1
xx = 0
ξ = 1e-5
while xx < Cron2[end]-Cron2[1]
    n+=1
    xx += (ξ*n)^2 
end
n

1292

In [13]:
steps = [ (ξ*i)^2 for i in 1:n ]
steps[1], steps[end]

(1.0000000000000002e-10, 0.00016692640000000002)

In [14]:
Cronadapt = accumulate(+, steps, init = Cron2[1] )
Cronadapt[1], Cronadapt[end]

(0.1245399699154478, 0.1965130908154476)

In [15]:
function yas(t, xo, yo, vo)
    
    cosϕ = xo / sqrt( xo^2 + yo^2 )
    sinϕ = yo / sqrt( xo^2 + yo^2 )
    atnx = -ħ*t / (2m*σ₀^2)
    sinatnx = atnx / sqrt( atnx^2 + 1)
    cosatnx =    1 / sqrt( atnx^2 + 1)
    #cosϕt = cosϕ * cosatnx - sinϕ * sinatnx
    sinϕt = sinϕ*cosatnx + cosϕ*sinatnx
    
    σ₀t = sqrt( σ₀^2 + ( ħ*t / (2*m*σ₀) )^2 )
    
    vo*t + sqrt(xo^2 + yo^2) * σ₀t/σ₀ * sinϕt
end

yas (generic function with 1 method)

In [29]:
function modelsolver2(Np = 10)
    
    Nt = length(Cronadapt)
    xo = [ init() for i = 1:Np ]
    #xo = vcat( myrng(a2, a1, Np), myrng(b1, b2, Np) )
    
    xcoord = copy(xo)
    ycoord = zeros(Np)
    
    for i in 2:Nt, j in 1:Np
        xcoord[j] = rk4(Uₓ, Cronadapt[i], xcoord[j], steps[i] )
    end
    
    for (i, x) in enumerate(xo)
        
        y0 = randn()*yh
        v0 = 0 #randn()*σv*1e-4
        
        ycoord[i] = yas(Cronadapt[end], x, y0, v0)
    end
    
    xcoord, ycoord
end

modelsolver2 (generic function with 2 methods)

In [33]:
@time X, Y = modelsolver2(2000);

345.404281 seconds (776.54 M allocations: 14.188 GiB, 0.44% gc time)


In [35]:
@gp "set title 'Impacts on the screen'" xlab="X, mm" ylab="Y, mm"# "set xrange [-1:1]"
@gp :- 1000X 1000Y "with points notit pt 7 ps 0.5 lc rgb 'black'"

GNUPLOT (default) -> gnuplot> plot  
GNUPLOT (default) ->                ^
GNUPLOT (default) ->          line 32177: function to plot expected


## Распараллелить

https://habr.com/ru/post/455846/

In [ ]:
using Distributed
addprocs(2)

In [ ]:
# animation
frame = 0
for i in 1:80
    global frame += 1
    @gp :- frame x psi[:, i] "with lines notit lw 2 lc rgb 'black'" :-
end
@gp


save(term="gif animate size 480,360 delay 15", output="aanimation.gif")